In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

# Import data
segmentData = pd.read_csv("./data/segments.csv")

# Remove irrelevent columns
segmentColumnsToRemove = ['Unnamed: 0', 'track_number', 'total_tracks', 'loudness_end', 'confidence', 'album_title', 'album_artist'] #'track_title'
segmentData.drop(columns = segmentColumnsToRemove, axis = 1, inplace = True)

# Convert Track Title to unique number
segmentTrackTitles = segmentData['track_title'].astype('category') # Extract the song title column
segmentDataCodes = segmentTrackTitles.cat.codes # Assign each song title to a unique number
segmentData['track_title'] = segmentDataCodes # Replace track title with unique number

# Initialise output matrix
allSongEuclidianTimbreHeadings = ['song_timbre', 'song_timbre_start', 'song_timbre_end', 'loudness_start', 'loudness_end']
allSongEuclidianTimbre = pd.DataFrame(np.zeros((max(segmentDataCodes), 5)), columns = allSongEuclidianTimbreHeadings) 

for song in range(0, max(segmentDataCodes)):
    songSegmentData = segmentData.loc[segmentData['track_title'] == song] # Extract segments for one song

    # Find number of segments in first 10 secs of song
    time = 0
    numStartSegments = 0
    while time < 10:
        time += songSegmentData.iloc[numStartSegments,1]
        numStartSegments += 1

    # Find number of segments in last 10 secs of song
    time = 0
    numEndSegments = 0
    while time < 10:
        time += songSegmentData.iloc[-numStartSegments,1]
        numEndSegments += 1

    # Timbre
    timbreStr = songSegmentData['timbre'] # Extract Timbre column vector from segmentData
    timbre = pd.DataFrame(np.zeros((12, len(timbreStr)))) # Initialise timbre matrix
    # Separate out each value of the Timbre column vector, and save to a column of the timbre matrix (then transpose)
    x=0
    for row in timbreStr:
        timbre[x] = pd.DataFrame(row.split(', '))
        x+=1
    timbre = timbre.transpose()
    # Remove the square brackets from start and end of the data rows
    timbre[0] = timbre[0].str[1:]
    timbre[11] = timbre[11].str[:-1]

    timbre = timbre.astype('float64') # Convert timbre matrix 

    # Weight timbre dimensions in order accourding to importance
    weightedTimbre = pd.DataFrame(np.zeros((len(timbre), 12))) 
    for x in range(1,12):
        weightedTimbre[x] = timbre[x]*(0.9**x)

    # For each row of timbre values find the euclidean distance of the 12 variables
    euclidianTimbre = pd.DataFrame(np.zeros((1, len(timbre))))
    for x in range(len(timbre)):
        euclidianTimbre[x] = np.linalg.norm(weightedTimbre.loc[x,:])
    euclidianTimbre = euclidianTimbre.transpose()

    # Normalise the data between zero and one
    euclidianTimbre = pd.DataFrame(MinMaxScaler(feature_range=(0,1)).fit_transform(euclidianTimbre))
    # euclidianTimbre.plot.line()

    # Average for euclidian distance for song
    euclidianTimbreSong = float(np.mean(euclidianTimbre, axis=0))
    # Average for the first 10 secs of the song
    euclidianTimbreSongStart = float(np.mean(euclidianTimbre.head(numStartSegments), axis=0))
    # Average for the last 10 secs of the song
    euclidianTimbreSongEnd = float(np.mean(euclidianTimbre.tail(numEndSegments), axis=0))

    # Loudness
    songLoudness = songSegmentData['loudness_max']
    # Average for the first 10 secs of the song
    loudnessSongStart = float(np.mean(songLoudness.head(numStartSegments), axis=0))
    # Average for the last 10 secs of the song
    loudnessSongEnd = float(np.mean(songLoudness.tail(numEndSegments), axis=0))

    allSongEuclidianTimbre.loc[song,:] = [euclidianTimbreSong, euclidianTimbreSongStart, euclidianTimbreSongEnd, loudnessSongStart, loudnessSongEnd] # Export values

allSongEuclidianTimbre

,song_timbre,song_timbre_start,song_timbre_end,loudness_start,loudness_end
0,0.349316,0.574728,0.350356,-16.800205,-12.889394
1,0.204448,0.264389,0.531644,-29.485143,-32.739208
2,0.262504,0.270538,0.215226,-14.189345,-7.432419
3,0.258499,0.279182,0.225166,-26.182976,-25.563583
4,0.274227,0.300487,0.258961,-26.984295,-26.162094
5,0.228498,0.231823,0.210949,-26.191415,-21.417883
6,0.231367,0.296660,0.234616,-27.258867,-19.973823
7,0.216832,0.251505,0.188220,-25.309659,-20.996110
8,0.228887,0.219365,0.186464,-30.568273,-24.050768
9,0.255767,0.277388,0.210195,-24.383714,-13.161430
